Import the library

In [ ]:
import os
import math
import skimage
import imageio

import skimage.filters as skfilters
import matplotlib.pyplot as plt

from scipy import ndimage
from scipy.signal import convolve2d
from scipy import misc
import numpy as np

define function mapmm

In [ ]:
def mapmm(image):
    # Equalization
    fimage = skimage.img_as_float(image)
    mina = 0.0
    maxa = 1.0
    minx = np.amin(fimage)
    maxx = np.amax(fimage)
    
    fimage = (fimage - minx) / (maxx-minx) * (maxa-mina) + mina

    return fimage


1: define functioin imblurgauss

In [ ]:
def imblurgauss(image, s):
    blurgauss = skfilters.gaussian(image, sigma=s,mode = 'nearest',truncate=2.0, multichannel=True)
    return blurgauss

2: define function imblurlens

In [ ]:
# define fucntion fspecial(shape = disk)
def fspecial_disk(radius):
    rad   = radius
    crad  = math.ceil(rad-0.5)
    x = np.arange(-crad, crad + 1)
    y = np.arange(-crad, crad + 1)
    xx,yy = np.meshgrid(x, y, sparse = True)
    


    maxxy = np.maximum(np.absolute(xx),np.absolute(yy))
    minxy = np.minimum(np.absolute(xx),np.absolute(yy))

    a = (rad**2 <  (maxxy+0.5)**2 + (minxy-0.5)**2) 
    b = (rad**2 >= (maxxy+0.5)**2 + (minxy-0.5)**2)    
    m1 = (a * 1 * (minxy-0.5) + b * 1 *np.sqrt((rad**2 - np.square(maxxy + 0.5)).astype(complex))).real
    
    a1 = (rad**2 >  (maxxy-0.5)**2 + (minxy+0.5)**2)
    b1 = (rad**2 <= (maxxy-0.5)**2 + (minxy+0.5)**2)    
    m2 = (a1 * 1 * (minxy+0.5) + b1 * 1 * np.sqrt((rad**2 - np.square(maxxy - 0.5)).astype(complex))).real

    
    a2 = (rad**2 <  (maxxy+0.5)**2 + (minxy+0.5)**2) 
    b2 = (rad**2 >= (maxxy-0.5)**2 + (minxy-0.5)**2)
#     sgrid = (rad^2*(0.5*(asin(m2/rad) - asin(m1/rad)) + ...
#             0.25*(sin(2*asin(m2/rad)) - sin(2*asin(m1/rad)))) - ...
#             (maxxy-0.5).*(m2-m1) + (m1-minxy+0.5)) ...
#             .*((((rad^2 < (maxxy+0.5).^2 + (minxy+0.5).^2) & ...
#             (rad^2 > (maxxy-0.5).^2 + (minxy-0.5).^2)) | ...
#             ((minxy==0)&(maxxy-0.5 < rad)&(maxxy+0.5>=rad))));
    f1 = (0.5*(np.arcsin(m2/rad) - np.arcsin(m1/rad)) + 0.25*(np.sin(2*np.arcsin(m2/rad)) - np.sin(2*np.arcsin(m1/rad))))
    first = (rad**2*f1 - (maxxy-0.5)*(m2-m1) + (m1-minxy+0.5))
    second = (((a2 & b2) | ((minxy==0)&(maxxy-0.5 < rad)&(maxxy+0.5>=rad))))

    sgrid = second * 1 * first
    
    a3 = (rad**2 >  (maxxy+0.5)**2 + (minxy+0.5)**2)

    sgrid = sgrid + a3 * 1    
    sgrid[crad,crad] = min(np.pi*rad**2,np.pi/2);
    
    
    if ((crad>0) and (rad > crad-0.5) and (rad**2 < (crad-0.5)**2+0.25)):
        
        m1  = sqrt(rad**2 - (crad - 0.5)**2)
        m1n = m1/rad
        sg0 = 2*(rad**2*(0.5*np.arcsin(m1n) + 0.25*np.sin(2*np.arcsin(m1n)))-m1*(crad-0.5))
        sgrid[2*crad,crad]     = sg0
        sgrid[crad,2*crad]     = sg0
        sgrid[crad,0]          = sg0
        sgrid[0,crad]          = sg0
        sgrid[2*crad-1,crad]   = sgrid(2*crad-1,crad) - sg0
        sgrid[crad,2*crad-1]   = sgrid(crad,2*crad-1) - sg0
        sgrid[crad,1]          = sgrid(crad,1)        - sg0
        sgrid[1,crad]          = sgrid(1,crad)        - sg0
    
    
    sgrid[crad,crad] = min(sgrid[crad,crad],1);
    
    h = sgrid/np.sum(sgrid);
    return h


    
def imblurlens(image, r):
    
    disk = fspecial_disk(r)
    ims = []
    for d in range(3):
        im_conv_d = convolve2d(image[:,:,d], disk, mode='same', boundary='fill')
        
        ims.append(im_conv_d)

    im_conv = np.stack(ims, axis=2).astype("float64")

    return im_conv

    

In [ ]:
# read images names
basepath = 'ref_img/'
examples_path = 'results1/'

# read and display original image
image = skimage.io.imread(fname=basepath + 'I01.BMP')

image = mapmm(image)

# viewer = ImageViewer(image)
# viewer.show()

if not os.path.exists(examples_path):
    # if not exist, make directory
    os.makedirs(examples_path)

im = plt.imread(basepath + 'I01.BMP')
# im = ndimage.imread(basepath + 'I01.BMP')
im = mapmm(im)


In [ ]:
# #1 gaussian blur

levels = [0.1, 0.2, 0.5, 0.7, 1, 1.5, 2, 3, 5]
for i in range(len(levels)):
    distort_I = imblurgauss(image, levels[i]);
    distort_I = mapmm(distort_I)
#     data = 255 * distort_I
#     distort_I = data.astype(np.uint8)
    imageio.imwrite(examples_path + 'distorted_gblur_' + str(i+1) + '.bmp', distort_I)

In [ ]:
# #2 lens blur
levels = [0.5, 1, 1.5, 2, 3, 4, 5, 6, 7, 8]
for i in range(len(levels)):
    distort_I = imblurlens(im, levels[i])
    distort_I = mapmm(distort_I)
#     data = 255 * distort_I
#     distort_I = data.astype(np.uint8)
    imageio.imwrite(examples_path + 'distorted_lblur_' + str(i+1) + '.bmp', distort_I)

